# Identification of Best Model 

Clean up code and emove unnecessary imports 


In [59]:
import pickle
import math

import pandas as pd
import numpy as np


import Utilities
import importlib
importlib.reload(Utilities)

import MLUtilities
importlib.reload(MLUtilities)

from sklearn.pipeline import Pipeline


# from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MinMaxScaler, PowerTransformer, RobustScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import  OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import r2_score, classification_report, confusion_matrix, roc_auc_score, accuracy_score, balanced_accuracy_score, precision_score

from sklearn.ensemble import RandomForestClassifier


# from hyperopt.pyll import scope
# from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK

from hyperopt import hp,Trials,fmin,tpe,STATUS_OK

import xgboost as xgb

from imblearn.over_sampling import SMOTENC

def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1],idx-1
    else:
        return array[idx],idx

## Load and Setup Verification data

# Settings for Machine Learning Model 

Modified code to exclude all < 18 yo patients.

Models based on first set of vital signs taken in ED. 

- DataSet 01: BP Systolic, Level of Consciousness, Pulse Rate (BPM), Respiration,  SpO2,  Temperature (Degrees C), O2 Flow
    + 01_a : Require age 65+
    + 01_b : 18 to 65 
- DataSet 02: DataSet 1 excluding O2 Flow 
    + 02_a : Require age 65+
    + 02_b : 18 to 65 
- DataSet 11: DataSet 1 plus Gender 
    + 11_a : Require age 65+
    + 11_b : 18 to 65 
- DataSet 12: DataSet 2 plus Gender 
    + 12_a : Require age 65+
    + 12_b : 18 to 65 
- DataSet 21: DataSet 1 plus Gender and Age
    + 21_a : Require age 65+
    + 21_b : 18 to 65 
- DataSet 22: DataSet 2 plus Gender and Age
    + 22_a : Require age 65+
    + 22_b : 18 to 65 

In [12]:
# Sepsis HeartFailure PE Pneumonia COPD UTI
# SepsisPneumonia
Diagnosis =   "Sepsis"
# Diagnosis = 'SepsisPneumonia'
# Diagnosis = 'Admission'
# Diagnosis = 'GenMed'

DiagnosisString=Diagnosis+'Flag'


# DataSet = 'DataSet3a'

# MLModel = 'randomforest'

print(Diagnosis,DiagnosisString)

Sepsis SepsisFlag


In [13]:
start_date =  '2023-01-01'  #dates[Facility][0]
end_date   =  '2024-01-01' #dates[Facility][1]

select_start_date = '2023-01-01'

DataReasons = "Training"

Data_Storage_File = 'DataFiles\Emergency_IPInfo_Complete_Numeric_Numeric_{}_{}_{}_Stage_01.pkl'.format(start_date,end_date,DataReasons)
print(Data_Storage_File)


with open(Data_Storage_File, 'rb') as file:
    Emergency_IPInfo_Complete_Numeric = pd.read_pickle(file)

DataFiles\Emergency_IPInfo_Complete_Numeric_Numeric_2023-01-01_2024-01-01_Training_Stage_01.pkl


In [14]:
Emergency_IPInfo_Complete_Numeric = Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION>17].copy()

Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(0,17),'age_range']='0-17'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(18,33),'age_range']='18-33'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(34,48),'age_range']='34-48'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(49,64),'age_range']='49-64'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(65,78),'age_range']='65-78'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(79,98),'age_range']='79-98'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(98,120),'age_range']='98+'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(121,200),'age_range']='Unknown'

Emergency_IPInfo_Complete_Numeric['IndiginousFlag'] = 0  # np.where((Emergency_IPInfo_Complete_Numeric.IndiginousStatus.isna()|Emergency_IPInfo_Complete_Numeric.IndiginousStatus=='Not Aboriginal-TSI'),0,1)
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.IndiginousStatus.isin(['Aboriginal','Aboriginal and TSI', 'TSI']),'IndiginousFlag'] = 1


# DiagnosisString='SepsisFlag'

if Diagnosis == "SepsisPneumonia":
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric.SepsisFlag==1)|(Emergency_IPInfo_Complete_Numeric.PneumoniaFlag==1), DiagnosisString] = 1

if Diagnosis == 'Admission':
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric['InitialIPLoS']>24), DiagnosisString] = 1
    
    if DataSet == 'DataSet5':
        Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
        Emergency_IPInfo_Complete_Numeric.loc[ (Emergency_IPInfo_Complete_Numeric['InitialIPLoS']>6), DiagnosisString] = 1


if Diagnosis == 'GenMed':
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    for g in GenMedList:
        Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric.GROUPCODE.str.contains(g))  & (Emergency_IPInfo_Complete_Numeric['InitialIPLoS']>6), DiagnosisString] = 1
   


In [15]:
importlib.reload(MLUtilities)
Emergency_IPInfo_Complete_Numeric = MLUtilities.setNumeric(Emergency_IPInfo_Complete_Numeric)

In [23]:
n_est=55


numeric_transformer = Pipeline(steps=[
        ('scaler', MinMaxScaler()),  # StandardScaler #MinMaxScaler # RobustScaler
        # ('scaler', RobustScaler(quantile_range=((25.0,75.0)))),  # StandardScaler #MinMaxScaler # RobustScaler
        #('skew', FunctionTransformer(log_transform))
    ])

space4xgbsepsis = {
    'random_state': n_est, 
    'max_depth': hp.choice('max_depth', np.arange(1,30, 1, dtype=int)), 
    'n_estimators': hp.choice('n_estimators', np.arange(1, 200, 1, dtype=int)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),  # New 
    'subsample':hp.uniform('subsample', 0.5, 1.),     # New
     }

space4rf = {
    'random_state': n_est,
    'class_weight': 'balanced',
    'max_depth':  None, #    hp.choice('max_depth', np.arange(5, 30, 1, dtype=int)),
    #'max_features':    #hp.choice('max_features', np.arange(2, 10, 1, dtype=int)),
    'n_estimators': hp.choice('n_estimators', np.arange(100, 500, 10, dtype=int))
}



In [25]:
def hyperparameter_tuning_rf_sepsis(trial_params):
        
    classifier = RandomForestClassifier(**trial_params)

    model = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", classifier)
        ])
    model.fit(X_train, np.ravel(y_train,order="c"))
    predictions_valid = model.predict_proba(X_valid)
    ypreds = np.delete(predictions_valid,[0],1)
    
    fpr, tpr, thresholds = roc_curve(y_valid, ypreds)
    val,idx = find_nearest(tpr,0.85)
    threshold =   np.interp(0.85,tpr,thresholds)    #thresholds[idx]
    false_positive =  np.interp(0.85,tpr,fpr)      # fpr[idx]
    # redo predictions with new threshold 
    y_select_2 = np.where(ypreds<threshold,0,1)
    auc_score = roc_auc_score(y_valid, ypreds, multi_class="ovr", average="weighted")
    accuracy  =accuracy_score(y_valid, y_select_2)
    balanced_accuracy = balanced_accuracy_score (y_valid, y_select_2)
    recall  =recall_score(y_valid, y_select_2)
    precision = precision_score(y_valid, y_select_2)
    f1 = f1_score(y_valid, y_select_2)
    

    
    scores = {"falsePositive":false_positive,
              "accuracy":accuracy,
              "precision":precision,
              "recall":recall,
              "precision":precision,
              "f1":f1,
              "auc":auc_score,
              "threshold":threshold}

    return {'status': STATUS_OK, 'loss': false_positive, 'model': model, 'attachments':scores}


def hyperparameter_tuning_xgb_sepsis(trial_params):
            
    classifier = xgb.XGBClassifier(**trial_params)

    model = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", classifier)
        ])
    
    model.fit(X_train, np.ravel(y_train,order="c"))
    predictions_valid = model.predict_proba(X_valid)
    ypreds = np.delete(predictions_valid,[0],1)
    
    fpr, tpr, thresholds = roc_curve(y_valid, ypreds)
    val,idx = find_nearest(tpr,0.85)
    threshold =   np.interp(0.85,tpr,thresholds)    #thresholds[idx]
    false_positive =  np.interp(0.85,tpr,fpr)      # fpr[idx]
    # redo predictions with new threshold 
    y_select_2 = np.where(ypreds<threshold,0,1)
    auc_score = roc_auc_score(y_valid, ypreds, multi_class="ovr", average="weighted")
    accuracy  =accuracy_score(y_valid, y_select_2)
    balanced_accuracy = balanced_accuracy_score (y_valid, y_select_2)
    recall  =recall_score(y_valid, y_select_2)
    precision = precision_score(y_valid, y_select_2)
    f1 = f1_score(y_valid, y_select_2)
    


    scores = {"falsePositive":false_positive,
              "accuracy":accuracy,
              "precision":precision,
              "recall":recall,
              "precision":precision,
              "f1":f1,
              "auc":auc_score,
              "threshold":threshold}

    return {'status': STATUS_OK, 'loss': false_positive, 'model': model, 'attachments':scores}

In [64]:
for DataSet in ['DataSet01',#'DataSet01_a','DataSet01_b','DataSet02','DataSet02_a','DataSet02_b',
                #'DataSet11',#'DataSet11_a','DataSet11_b','DataSet12','DataSet12_a','DataSet12_b',
                #'DataSet21',#'DataSet21_a','DataSet21_b','DataSet22','DataSet22_a','DataSet22_b',]:
]:
    
    n_trials = 2
    
    AnalysisVariable =['FirstBPSystolic', 'FirstLevelofConsciousness',
       'FirstPulseRateBPM', 'FirstRespiration', 'FirstSpO2',
       'FirstTemperatureDegreesC', DiagnosisString]

    if '1_' in DataSet:
        AnalysisVariable.append('FirstO2Flow')
    if '01' in DataSet:
        AnalysisVariable.append('FirstO2Flow')
    if '11' in DataSet:
        AnalysisVariable.append('FirstO2Flow')
    if '21' in DataSet:
        AnalysisVariable.append('FirstO2Flow')
    if ('DataSet1' in DataSet):
        AnalysisVariable.append('GENDERCODE')
    if ('DataSet2' in DataSet):
        AnalysisVariable.append('GENDERCODE')
        AnalysisVariable.append('AGEONADMISSION')

        
    if '_a' in DataSet:
        print("check")
        df = Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION>64].copy()
    elif '_b' in DataSet:
        df = Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION<65].copy()
    else: 
        df = Emergency_IPInfo_Complete_Numeric.copy()
        
        
        
    df.name='{} diagnosis detected from full Diagnosis list, updated'.format(Diagnosis)
    importlib.reload(MLUtilities)
    df = MLUtilities.setDefaults(df)

    print("\n{}".format(DataSet))
    print("Diagnosis being investigated = {}".format(DiagnosisString))
    print("Total Number of Cases = {}".format(len(df.index)))
    print("Number of {} Cases ".format(DiagnosisString),len(df[df[DiagnosisString]==1]))
    print(AnalysisVariable)
    
    df_temp = df[AnalysisVariable].copy()
    
    if 'GENDERCODE' in AnalysisVariable:
        df_temp['GENDERCODE'] = df_temp['GENDERCODE'].astype('category')
    
    # SMOTE does not work if NAN is allowed so int instead of Int 

    int_features = df_temp.select_dtypes(
        include=[ 'int64', 'Int64', 'int32']).columns

    for key in int_features:
        df_temp[key] = df_temp[key].astype('int64')
    
    y = df_temp[[DiagnosisString]]
    X = df_temp[AnalysisVariable].drop(DiagnosisString, axis=1).copy()
    x_vars = X.columns
    
    skew_vars = []

    
    numeric_features = X.drop(skew_vars, axis=1).select_dtypes(
        include=['float64', 'int64']).columns

    categorical_features = X.select_dtypes(
        include=['category', 'object']).columns
    
    category_values = []
    for i in categorical_features:
        tmp = pd.unique(df_temp[i]).tolist()
        tmp.sort()
        print(i,tmp)
        category_values.append(tmp)
    
    categorical_transformer = Pipeline(steps=[
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore',categories=category_values   ))])  # OrdinalEncoder() #OneHotEncoder(drop='first')

    preprocessor = ColumnTransformer(
            transformers=[
                ('numeric', numeric_transformer, numeric_features),
                ('categorical', categorical_transformer, categorical_features)
            ])

    def pip_f(reg):
        pip = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', reg)
        ])
        return(pip)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)
    
    if len(categorical_features)>0:
        smote = SMOTENC(random_state=42, categorical_features=[X_train.columns.get_loc(c) for c in categorical_features])
        X_train, y_train= smote.fit_resample(X_train, y_train)
        
        
    # n_trials = 2
    
    trialsRandomForest = Trials()
    trialsXGBoost = Trials()


    best_params_XGB = fmin(fn = hyperparameter_tuning_xgb_sepsis,
                        space = space4xgbsepsis,
                        algo = tpe.suggest,
                        max_evals = n_trials*10, 
                        trials = trialsXGBoost
                        )

    best_params_RandomForest = fmin(fn=hyperparameter_tuning_rf_sepsis,
                        space=space4rf,
                        algo=tpe.suggest,
                        max_evals=n_trials,
                        trials=trialsRandomForest
                        )
    
    scoreRandomForest =  [t['result']['loss'] for t in trialsRandomForest.trials]
    best_model_RandomForest=trialsRandomForest.results[np.argmin([r['loss'] for r in trialsRandomForest.results])]['model']


    scoreXGBoost = [t['result']['loss'] for t in trialsXGBoost.trials]
    best_model_XGBoost=trialsXGBoost.results[np.argmin([r['loss'] for r in trialsXGBoost.results])]['model']
    
    all_cols = np.concatenate([numeric_features])

    if len(categorical_features)>0:
        new_cat_cols =best_model_RandomForest['preprocessor'].transformers_[1][1].named_steps['encoder'].get_feature_names_out(categorical_features)
        all_cols = np.concatenate([numeric_features,new_cat_cols])


    xgbValues = []
    varStrings = ["falsePositive",  "threshold","accuracy", "precision", "recall","precision","f1", "auc"]
    scores = {}
    i=0
    for i in range(n_trials*10):
        scores = {}
        # if i < 2:
        scores['max_depth'] = trialsXGBoost.trials[i]['misc']['vals']['max_depth'][0]+1
        scores['n_estimators'] = trialsXGBoost.trials[i]['misc']['vals']['n_estimators'][0]+1
        scores['learning_rate'] = trialsXGBoost.trials[i]['misc']['vals']['learning_rate'][0]+1
        scores['subsample'] = trialsXGBoost.trials[i]['misc']['vals']['subsample'][0]+1
        # scores['seed'] = trialsXGBoost.trials[i]['misc']['vals']['seed'][0]
            # 
            # 'n_estimators'
        for c in varStrings:
            # print(c)
            key = "ATTACH::{}::{}".format(i,c)
            scores[c] = trialsXGBoost.attachments[key]
            scores["i"] = i
        xgbValues.append(scores)
    # print (scores)
    trials_XGB_df = pd.DataFrame.from_dict(xgbValues)

    rfValues = []
    varStrings = ["falsePositive", "accuracy", "precision", "recall","precision","f1", "auc",]
    scores = {}
    i=0
    for i in range(n_trials):
        scores = {}
        # if i < 2:
        # scores['max_depth'] = trialsRandomForest.trials[i]['misc']['vals']['max_depth'][0]+1
        scores['n_estimators'] = trialsRandomForest.trials[i]['misc']['vals']['n_estimators'][0]+1

            # 
            # 'n_estimators'
        for c in varStrings:
            # print(c)
            key = "ATTACH::{}::{}".format(i,c)
            scores[c] = trialsRandomForest.attachments[key]
            scores["i"] = i
        rfValues.append(scores)

    trials_rf_df = pd.DataFrame.from_dict(rfValues)
    
    ObjectsToSaveRF = [best_model_RandomForest, best_model_XGBoost, all_cols, X_valid, y_valid,trials_rf_df,trials_XGB_df]

    # timestr =  str(datetime.datetime.now().strftime('%Y_%m_%d'))
    # Data_Storage_File = 'Results/GenMed_SepsisPathway_{}_{}_FeatureImportances.pkl'.format(DataSet,DiagnosisString)

    Data_Storage_File = 'Results/Nov_2024_{}_{}_BestFit_Models_Pathway_ED.pkl'.format(DiagnosisString,DataSet)
    print(Data_Storage_File)

    with open(Data_Storage_File, 'wb') as file:  
        pickle.dump(ObjectsToSaveRF, file)



DataSet01
Diagnosis being investigated = SepsisFlag
Total Number of Cases = 277021
Number of SepsisFlag Cases  2733
['FirstBPSystolic', 'FirstLevelofConsciousness', 'FirstPulseRateBPM', 'FirstRespiration', 'FirstSpO2', 'FirstTemperatureDegreesC', 'SepsisFlag', 'FirstO2Flow']
100%|██████████| 2/2 [00:53<00:00, 26.80s/trial, best loss: 0.446570534017649] 
Results/Nov_2024_SepsisFlag_DataSet01_BestFit_Models_Pathway_ED.pkl


In [66]:
# trialsXGBoost.results[0]['model'][1].get_params()

In [67]:
all_cols = np.concatenate([numeric_features])

if len(categorical_features)>0:
    new_cat_cols =best_model_RandomForest['preprocessor'].transformers_[1][1].named_steps['encoder'].get_feature_names_out(categorical_features)
    all_cols = np.concatenate([numeric_features,new_cat_cols])

In [68]:
xgbValues = []
varStrings = ["falsePositive",  "threshold","accuracy", "precision", "recall","precision","f1", "auc"]
scores = {}
i=0
for i in range(n_trials*10):
    scores = {}
    # if i < 2:
    scores['max_depth'] = trialsXGBoost.trials[i]['misc']['vals']['max_depth'][0]+1
    scores['n_estimators'] = trialsXGBoost.trials[i]['misc']['vals']['n_estimators'][0]+1
    scores['learning_rate'] = trialsXGBoost.trials[i]['misc']['vals']['learning_rate'][0]+1
    scores['subsample'] = trialsXGBoost.trials[i]['misc']['vals']['subsample'][0]+1
    # scores['seed'] = trialsXGBoost.trials[i]['misc']['vals']['seed'][0]
        # 
        # 'n_estimators'
    for c in varStrings:
        # print(c)
        key = "ATTACH::{}::{}".format(i,c)
        scores[c] = trialsXGBoost.attachments[key]
        scores["i"] = i
    xgbValues.append(scores)
# print (scores)
trials_XGB_df = pd.DataFrame.from_dict(xgbValues)

rfValues = []
varStrings = ["falsePositive", "accuracy", "precision", "recall","precision","f1", "auc",]
scores = {}
i=0
for i in range(n_trials):
    scores = {}
    # if i < 2:
    # scores['max_depth'] = trialsRandomForest.trials[i]['misc']['vals']['max_depth'][0]+1
    scores['n_estimators'] = trialsRandomForest.trials[i]['misc']['vals']['n_estimators'][0]+1

        # 
        # 'n_estimators'
    for c in varStrings:
        # print(c)
        key = "ATTACH::{}::{}".format(i,c)
        scores[c] = trialsRandomForest.attachments[key]
        scores["i"] = i
    rfValues.append(scores)

trials_rf_df = pd.DataFrame.from_dict(rfValues)

In [ ]:
ObjectsToSaveRF = [best_model_RandomForest, best_model_XGBoost, all_cols, X_valid, y_valid,trials_rf_df,trials_XGB_df]

# timestr =  str(datetime.datetime.now().strftime('%Y_%m_%d'))
# Data_Storage_File = 'Results/GenMed_SepsisPathway_{}_{}_FeatureImportances.pkl'.format(DataSet,DiagnosisString)

Data_Storage_File = 'Results/Nov_2024_{}_{}_BestFit_Models_Pathway_ED.pkl'.format(DiagnosisString,DataSet)
print(Data_Storage_File)

with open(Data_Storage_File, 'wb') as file:  
    pickle.dump(ObjectsToSaveRF, file)